# Handling Large Multi-Modal Payloads in AgentCore Runtime

## Overview

This tutorial demonstrates how Amazon Bedrock AgentCore Runtime handles large payloads up to 100MB, including multi-modal content such as Excel files and images. AgentCore Runtime is designed to process rich media content and large datasets seamlessly.

### Tutorial Details

|Information| Details|
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Large Payload & Multi-Modal Processing|
| Agent type          | Single         |
| Agentic Framework   | Strands Agents |
| LLM model           | Anthropic Claude Haiku 4.5 |
| Tutorial components | Large File Processing, Image Analysis, Excel Data Processing |
| Tutorial vertical   | Data Analysis & Multi-Modal AI                                                   |
| Example complexity  | Intermediate                                                                     |
| SDK used            | Amazon BedrockAgentCore Python SDK|

### Key Features

* **Large Payload Support**: Process files up to 100MB in size
* **Multi-Modal Processing**: Handle Excel files, images, and text simultaneously
* **Data Analysis**: Extract insights from structured data and visual content
* **Base64 Encoding**: Secure transmission of binary data through JSON payloads

## Prerequisites

* Python 3.10+
* AWS credentials configured
* Docker running
* Sample Excel file and image for testing

In [1]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
sagemaker-studio 1.1.1 requires pydynamodb>=0.7.4, which is not installed.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.42.7 which is incompatible.
amazon-sagemaker-jupyter-ai-q-developer 1.2.8 requires numpy<=2.0.1, but you have numpy 2.3.5 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 requires numpy<2, but you have numpy 2.3.5 which is incompatible.
autogluon-multimodal 1.4.0 requires j

## Create Sample Data Files

Let's create sample Excel and image files to demonstrate large payload handling:

In [2]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os

# Create a large Excel file with sample sales data
np.random.seed(42)
data = {
    'Date': pd.date_range('2023-01-01', periods=1000, freq='h'),
    'Product': np.random.choice(['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y'], 1000),
    'Sales': np.random.randint(1, 1000, 1000),
    'Revenue': np.random.uniform(10.0, 5000.0, 1000),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 1000),
    'Customer_ID': np.random.randint(1000, 9999, 1000)
}

df = pd.DataFrame(data)
df.to_excel('large_sales_data.xlsx', index=False)

# Create a sample chart image
img = Image.new('RGB', (600, 500), color='white')
draw = ImageDraw.Draw(img)

# Draw a simple bar chart
products = ['Widget A', 'Widget B', 'Widget C', 'Gadget X', 'Gadget Y']
values = [250, 180, 320, 150, 280]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']

max_value = max(values)
bar_width = 120
start_x = 100

for i, (product, value, color) in enumerate(zip(products, values, colors)):
    x = start_x + i * (bar_width + 20)
    height = int((value / max_value) * 400)
    y = 500 - height
    
    # Draw bar
    draw.rectangle([x, y, x + bar_width, 500], fill=color)
    
    # Add labels (simplified without font)
    draw.text((x + 10, 510), product[:8], fill='black')
    draw.text((x + 10, y - 20), str(value), fill='black')

draw.text((300, 50), 'Sales Performance by Product', fill='black')
img.save('sales_chart.png')

# Check file sizes
excel_size = os.path.getsize('large_sales_data.xlsx') / (1024 * 1024)  # MB
image_size = os.path.getsize('sales_chart.png') / (1024 * 1024)  # MB

print(f"Excel file size: {excel_size:.2f} MB")
print(f"Image file size: {image_size:.2f} MB")
print(f"Total payload size: {excel_size + image_size:.2f} MB")

Excel file size: 0.05 MB
Image file size: 0.01 MB
Total payload size: 0.05 MB


## Create Multi-Modal Agent

Let's create an agent that can process both Excel files and images from large payloads:

In [3]:
%%writefile multimodal_data_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
import pandas as pd
import base64
import io
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# Initialize the model and agent
model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
    max_tokens=16000
)

agent = Agent(
    model=model,
    system_prompt="""
    You are a data analysis assistant that can process large Excel files and images.
    When given multi-modal data, analyze both the structured data and visual content,
    then provide comprehensive insights combining both data sources.
    """
)

@app.entrypoint
def multimodal_data_processor(payload, context):
    """
    Process large multi-modal payloads containing Excel data and images.
    
    Args:
        payload: Contains prompt, excel_data (base64), image_data (base64)
        context: Runtime context information
    
    Returns:
        str: Analysis results from both data sources
    """
    prompt = payload.get("prompt", "Analyze the provided data.")
    excel_data = payload.get("excel_data", "")
    image_data = payload.get("image_data", "")
    
    print(f"=== Large Payload Processing ===")
    print(f"Session ID: {context.session_id}")
    
    if excel_data:
        print(f"Excel data size: {len(excel_data) / 1024 / 1024:.2f} MB")
    if image_data:
        print(f"Image data size: {len(image_data) / 1024 / 1024:.2f} MB")
    print(f"Excel data {excel_data}")
    print(f"Image data {image_data}")
    print(f"=== Processing Started ===")
    # Decode base64 to bytes
    excel_bytes = base64.b64decode(excel_data)
    # Decode base64 to bytes
    image_bytes = base64.b64decode(image_data)
    
    # Enhanced prompt with data context
    enhanced_prompt = f"""{prompt}
    Please analyze both data sources and provide insights.
    """
    
    response = agent(
        [{
            "document": {
                "format": "xlsx",
                "name": "excel_data",
                "source": {
                    "bytes": excel_bytes
                }
            }
        },
        {
            "image": {
                "format": "png",
                "source": {
                    "bytes": image_bytes
                }
            }
        },
        {
            "text": enhanced_prompt
        }]
    )
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Writing multimodal_data_agent.py


## Setup Infrastructure and Deploy Agent

In [4]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="multimodal_data_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="multimodal_data_agent"
)

launch_result = agentcore_runtime.launch()

Entrypoint parsed: file=/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/11-AgentCore-runtime/03-advanced-concepts/03-handling-large-payloads/multimodal_data_agent.py, bedrock_agentcore_name=multimodal_data_agent
Configuring BedrockAgentCore agent: multimodal_data_agent


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/11-AgentCore-runtime/03-advanced-concepts/03-handling-large-payloads/Dockerfile
Generated .dockerignore: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/11-AgentCore-runtime/03-advanced-concepts/03-handling-large-payloads/.dockerignore
Setting 'multimodal_data_agent' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/11-AgentCore-runtime/03-advanced-concepts/03-handling-large-payloads/.bedrock_agentcore.yaml
🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Lo

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-multimodal_data_agent


Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-069e2d958e
Starting execution role creation process for agent: multimodal_data_agent
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-west-2-069e2d958e
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-069e2d958e
✓ Role created: arn:aws:iam::455933813601:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-069e2d958e
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-multimodal_data_agent
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::455933813601:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-069e2d958e
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: multimodal_data_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-069e2d958e
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-069e2d958e
Creating 

In [8]:
cat Dockerfile

FROM ghcr.io/astral-sh/uv:python3.12-bookworm-slim
WORKDIR /app

# Configure UV for container environment
ENV UV_SYSTEM_PYTHON=1 UV_COMPILE_BYTECODE=1



COPY requirements.txt requirements.txt
# Install from requirements file
RUN uv pip install -r requirements.txt




RUN uv pip install aws-opentelemetry-distro>=0.10.1


# Set AWS region environment variable

ENV AWS_REGION=us-west-2
ENV AWS_DEFAULT_REGION=us-west-2


# Signal that this is running in Docker for host binding logic
ENV DOCKER_CONTAINER=1

# Create non-root user
RUN useradd -m -u 1000 bedrock_agentcore
USER bedrock_agentcore

EXPOSE 8080
EXPOSE 8000

# Copy entire project (respecting .dockerignore)
COPY . .

# Use the full module path

CMD ["opentelemetry-instrument", "python", "-m", "multimodal_data_agent"]


In [5]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"Deployment status: {status}")

print(f"Final status: {status}")

Retrieved Bedrock AgentCore status for: multimodal_data_agent


Final status: READY


## Test Large Multi-Modal Payloads

Now let's test the agent with large payloads containing both Excel data and images:

In [6]:
import base64
import uuid
import json
from IPython.display import Markdown, display

# Encode files to base64
with open('large_sales_data.xlsx', 'rb') as f:
    excel_base64 = base64.b64encode(f.read()).decode('utf-8')

with open('sales_chart.png', 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

# Create large payload
large_payload = {
    "prompt": "Analyze the sales data from the Excel file and correlate it with the chart image. Provide insights on sales performance and trends.",
    "excel_data": excel_base64,
    "image_data": image_base64
}

session_id = str(uuid.uuid4())
print(f"📊 Processing large multi-modal payload...")
print(f"📋 Session ID: {session_id}")
print(f"📄 Excel size: {len(excel_base64) / 1024 / 1024:.2f} MB")
print(f"🖼️ Image size: {len(image_base64) / 1024 / 1024:.2f} MB")
print(f"📦 Total payload: {len(json.dumps(large_payload)) / 1024 / 1024:.2f} MB\n")

# Invoke agent with large payload
invoke_response = agentcore_runtime.invoke(
    large_payload,
    session_id=session_id
)
final_response = ""
for r in invoke_response['response']:
    final_response += r
response_data = final_response
display(Markdown(response_data))

📊 Processing large multi-modal payload...
📋 Session ID: ba810f50-3949-47f5-baf4-591d25397454
📄 Excel size: 0.06 MB
🖼️ Image size: 0.01 MB
📦 Total payload: 0.07 MB



"# Sales Performance Analysis: Data & Visual Insights\n\n## Overview\nBased on the Excel dataset (Jan 1 - Feb 11, 2023) and the sales performance chart, here are the comprehensive findings:\n\n---\n\n## Key Sales Metrics by Product\n\n### Sales Volume (from chart):\n- **Gadget X**: 250 units\n- **Widget C**: 180 units\n- **Widget B**: 320 units ⭐ (highest)\n- **Widget A**: 150 units\n\n### Detailed Analysis:\n\n#### 🥇 **Widget B - Top Performer**\n- **Sales Volume**: 320 units (highest)\n- **Market Position**: Clear leader in sales quantity\n- **Consistency**: Strong performance across regions\n- **Regional Strength**: Distributed well across East, North, South, and West regions\n\n#### 🥈 **Gadget X - Strong Second**\n- **Sales Volume**: 250 units\n- **Performance**: Consistent high performer\n- **Revenue Efficiency**: Generally good revenue per unit despite lower units than Widget B\n- **Market Appeal**: Steady demand throughout the period\n\n#### 🥉 **Widget C - Mid-Tier Performer**\n- **Sales Volume**: 180 units\n- **Performance**: Moderate sales with variable pricing\n- **Notable**: High volatility in revenue per unit (ranging from $16 to $4,839)\n\n#### ⚠️ **Widget A - Underperformer**\n- **Sales Volume**: 150 units (lowest)\n- **Concern**: Lagging significantly behind competitors\n- **Opportunity**: Potential growth area needing strategic focus\n\n---\n\n## Revenue Insights\n\n### From Data Analysis:\n- **Average Revenue per Unit**:\n  - Widget B: ~$2,573 (strong revenue conversion)\n  - Gadget X: ~$2,241 (reliable)\n  - Widget C: ~$2,108 (variable)\n  - Widget A: ~$2,094 (needs improvement)\n\n### Revenue Efficiency:\n- Despite Widget B having the most units, revenue consistency varies\n- Gadget X maintains stable revenue performance\n- Widget A needs pricing strategy review or product enhancement\n\n---\n\n## Regional Performance Trends\n\nAcross all regions (East, North, South, West):\n- **No single region dominates** - sales are well-distributed\n- **East & North** show slightly higher transaction volumes\n- **Seasonal/Daily Patterns**: Some variation by hour of day, suggesting time-based demand fluctuations\n\n---\n\n## Strategic Recommendations\n\n### 1. **Maximize Widget B Success**\n   - Dominant market leader—maintain inventory and distribution\n   - Investigate reasons for success for application to other products\n\n### 2. **Support Gadget X Growth**\n   - Second strongest performer with stable revenue\n   - Consider increased marketing to boost units sold\n\n### 3. **Revitalize Widget A**\n   - Currently underperforming at 150 units\n   - Conduct market research on customer preferences\n   - Consider pricing adjustments or product improvements\n\n### 4. **Stabilize Widget C Revenue**\n   - High volatility suggests inconsistent pricing or demand\n   - Standardize pricing strategy for predictability\n\n### 5. **Regional Expansion**\n   - All regions show balanced performance\n   - Opportunity for targeted regional campaigns to boost underperforming areas\n\n---\n\n## Conclusion\n\nWidget B clearly dominates the portfolio, but a balanced approach focusing on Gadget X's reliability and Widget A's turnaround potential would strengthen overall business performance. The data suggests **February showed potential for sustained growth**, with consistent customer engagement across all regions."

## Cleanup Resources

In [7]:
import boto3

# Clean up AWS resources
agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)

# Delete AgentCore Runtime
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# Delete ECR repository
ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Clean up local files
os.remove('large_sales_data.xlsx')
os.remove('sales_chart.png')

print("✅ Cleanup completed!")

✅ Cleanup completed!


# Congratulations!

You have successfully demonstrated handling large multi-modal payloads with Amazon Bedrock AgentCore Runtime!

## What you've learned:

### Large Payload Processing
* **100MB Support**: AgentCore Runtime can handle payloads up to 100MB
* **Base64 Encoding**: Secure transmission of binary data through JSON payloads
* **Efficient Processing**: Runtime optimized for large data processing

### Multi-Modal Capabilities
* **Excel Analysis**: Processing structured data from spreadsheets
* **Image Processing**: Analyzing visual content and charts
* **Combined Analysis**: Correlating insights from multiple data types

### Key Benefits
* **Rich Data Processing**: Handle complex, multi-format datasets
* **Scalable Architecture**: Runtime designed for large workloads
* **Tool Integration**: Custom tools for specialized data processing
* **Enterprise Ready**: Secure handling of sensitive business data

This demonstrates AgentCore Runtime's capability to handle enterprise-scale data processing tasks with multiple data modalities, making it ideal for complex business intelligence and data analysis applications.